In [7]:
import os
import json
import re
from groq import Groq

client = Groq(
    api_key="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",
)

# Modify the prompt to request JSON output
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": (
                "Explain the importance of fast language models. "
                "Provide the answer in JSON format with keys 'summary', 'benefits', and 'applications'."
            ),
        }
    ],
    model="llama3-8b-8192",
)

content = chat_completion.choices[0].message.content

# Use regex to extract JSON content in case there is extra text
json_match = re.search(r'\{.*\}', content, re.DOTALL)

if json_match:
    json_content = json_match.group()
    try:
        data = json.loads(json_content)
        print(json.dumps(data, indent=4))
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        print("Content was:", json_content)
else:
    print("No JSON content found in the response.")


{
    "summary": {
        "text": "Fast language models have revolutionized the field of natural language processing (NLP) by enabling efficient and accurate processing of vast amounts of text data. They have numerous applications in various industries and have become a crucial component of many AI-powered systems.",
        "importance": "Fast language models are important because they can process and generate human-like language at high speeds, making them useful for tasks that require quick responses or decisions."
    },
    "benefits": [
        {
            "name": "Improved Accuracy",
            "description": "Fast language models are trained on large datasets and can generate more accurate results than traditional language models."
        },
        {
            "name": "Faster Processing",
            "description": "They can process large amounts of text data quickly, making them suitable for real-time applications."
        },
        {
            "name": "Enhanced Us

In [ ]:
#LLM Inference Default

import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

url = "http://localhost:8000/chat/completions"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

data = {
    "messages": [
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    "model": "llama3.1-8b-8192",
    "service_name": "cerebras",
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
#LLM Inference Custom Parameters

import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

url = "http://localhost:8000/chat/completions"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

data = {
  "messages": [
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
  "model": "llama3.1-8b",
  "service_name": "cerebras",
  "temperature": 0.7,
  "max_tokens": 500,
  "top_p": 0.9,
  "stop": ["\n"]
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")